In [26]:
from sklearn.metrics import classification_report
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, \
    BatchNormalization, MaxPooling2D, Conv2D, Activation, Flatten
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing.image import load_img, \
    ImageDataGenerator

print("加载模块完成")

加载模块完成


In [22]:
def model_build(width: int,
                height: int,
                depth: int) -> Sequential:
    model = Sequential()

    # first
    model.add(Conv2D(input_shape=(width,
                                  height,
                                  depth),
                     filters=64,
                     kernel_size=(3, 3),
                     padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization())

    # second
    model.add(Conv2D(64, kernel_size=(3, 3),
                     padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    # third
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization())

    # four
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    # finally
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(256))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dense(2))
    model.add(Activation("softmax"))

    return model

In [6]:
train_dir = "data/training_set"
test_dir = "data/test_set"
data_gen = ImageDataGenerator(rescale=1.0 / 255)
train_data = data_gen.flow_from_directory(train_dir,
                                          target_size=(32, 32))
test_data = data_gen.flow_from_directory(test_dir,
                                         target_size=(32, 32))
print("数据加载完成")
print(train_data.class_indices)


Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.
数据加载完成
{'cats': 0, 'dogs': 1}


In [27]:
epochs = 20
model = model_build(32, 32, 3)
sgd = SGD(learning_rate=0.01,
          momentum=0.05,
          nesterov=True,
          )
adam = Adam(learning_rate=0.01)
model.compile(loss="binary_crossentropy",
              optimizer=adam,
              metrics=["acc"])
print("模型编译成功")
model.summary()

模型编译成功
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_41 (Activation)   (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_23 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
activation_42 (Activation)   (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
batch_normalization_24 (Batc (None, 16, 16, 64)

In [28]:
history = model.fit(train_data,
                    epochs=epochs,
                    validation_data=test_data,
                    batch_size=64)


Epoch 1/20
251/251 [==============================] - 13s 47ms/step - loss: 0.7408 - acc: 0.5755 - val_loss: 0.8085 - val_acc: 0.5175